In [129]:
import math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torchtext
import matplotlib.pyplot as plt
from torchinfo import summary

In [130]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = get_tokenizer("basic_english")
class EEPD_Dataset(Dataset):
    def __init__(self):
        
        data = 'data/WASSA23_conv_level_with_labels_train.tsv'

        xy = pd.read_table(data, header=0)
        new_col = []
        for names in xy.columns:
            new_col.append(names.strip())
        xy.columns = new_col
        x, y = xy.loc[:, 'text'], xy.loc[:, ['EmotionalPolarity', 'Emotion', 'Empathy']]
        y = np.array(y)


        padd_to_tensor = torchtext.transforms.ToTensor(padding_value=0)
        vocab = build_vocab_from_iterator(self.yield_tokens(x), specials=["<unk>"])
        vocab.set_default_index(vocab["<unk>"])
        train_x, train_y = [], []
        for text, label in zip(x, y):
            x_tokens = vocab(tokenizer(text))
            y_label = np.array(label)
            train_x.append(x_tokens)
            train_y.append(y_label)
        train_x = padd_to_tensor(train_x)
        
        self.text = torch.tensor(train_x, dtype=torch.float32).to(device)
        self.label = torch.tensor(train_y).to(device)
        self.n_samples = x.shape[0]


    def __getitem__(self, index):
        return self.text[index], self.label[index]
    
    def __len__(self):
        return self.n_samples
    
    def yield_tokens(self, text_data):
        for text in text_data:
            yield tokenizer(text)



In [131]:
class DeepNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_classes):
        super(DeepNN,self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(hidden_size, output_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out


In [134]:
dataset = EEPD_Dataset()
#train, test split
generator = torch.Generator().manual_seed(42)
train_set, test_set = random_split(dataset=dataset, lengths=[0.8, 0.2], generator=generator)

learning_rate = 0.01
n_samples = len(train_set)
n_epochs = 32
n_input = len(train_set[0][0])
n_hidden = 100
n_classes = 3
n_batch = 64
n_iterations = math.ceil(n_samples/n_batch)

print(f'n_samples: {n_samples} | n_iterations/batch: {n_iterations}')

train_dataloader = DataLoader(dataset=train_set, batch_size=n_batch, shuffle=True)
test_dataloader = DataLoader(dataset=test_set, batch_size=1, shuffle=False)

data_iter = iter(train_dataloader)

# for i in range(1):
#     feature,label = next(data_iter)
# feature,label

n_samples: 7021 | n_iterations/batch: 110


C:\Users\AttahiruJibril\AppData\Local\Temp\ipykernel_6172\2203063665.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.text = torch.tensor(train_x, dtype=torch.float32).to(device)


In [ ]:
data_iter = iter(test_dataloader)
x, y = next(data_iter)
x,y

In [140]:
model = DeepNN(n_input, n_hidden, n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(summary(model, batch_dim=n_batch))
for epoch in range(n_epochs):
    for i, (input, label) in enumerate(train_dataloader):
        #forward
        output = model(input)
        loss = criterion(output, label)
        
        #backward
        optimizer.zero_grad
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print(f'epoch: {epoch+1}/{n_epochs} | step: {i+1}/{n_iterations} | loss: {loss.item():.4f}')
    
#test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i, (input, label) in enumerate(train_dataloader):
        outputs = model(input)
        # _, predictions = torch.max(outputs, 1)
        # n_samples += label.shape[0]
    #     n_correct = (predictions == label).sum().item()
    # accuracy = 100.0 * n_correct /n_samples
    # print(f'accuracy: {accuracy}')
    # print(outputs)


Layer (type:depth-idx)                   Param #
DeepNN                                   --
├─Linear: 1-1                            13,400
├─ReLU: 1-2                              --
├─Linear: 1-3                            10,100
├─Linear: 1-4                            303
Total params: 23,803
Trainable params: 23,803
Non-trainable params: 0
epoch: 1/32 | step: 100/110 | loss: 6.3765
epoch: 2/32 | step: 100/110 | loss: 5.9431
epoch: 3/32 | step: 100/110 | loss: 6.2992
epoch: 4/32 | step: 100/110 | loss: 6.4690
epoch: 5/32 | step: 100/110 | loss: 8.1553
epoch: 6/32 | step: 100/110 | loss: 6.3326
epoch: 7/32 | step: 100/110 | loss: 7.8228
epoch: 8/32 | step: 100/110 | loss: 6.8815
epoch: 9/32 | step: 100/110 | loss: 9.0216
epoch: 10/32 | step: 100/110 | loss: 7.8409
epoch: 11/32 | step: 100/110 | loss: 9.4121
epoch: 12/32 | step: 100/110 | loss: 8.3132
epoch: 13/32 | step: 100/110 | loss: 7.7587
epoch: 14/32 | step: 100/110 | loss: 12.4483
epoch: 15/32 | step: 100/110 | loss: 7.4225
